### base_model / model

In [1]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import neural_structured_learning as nsl
import time, os
policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
tf.keras.mixed_precision.experimental.set_policy(policy)

In [2]:
inputs = {'image': tf.keras.Input((96,96,3), name='image'),}
model_body = tf.keras.applications.Xception(input_shape=(96,96,3), 
            include_top=False)
for layer in model_body.layers:
    layer.trainable = True
x = model_body(inputs['image'])
x = tf.keras.layers.GlobalAveragePooling2D()(x)
# x = tf.keras.layers.Dense(128, name='dense_logits_128')(x)
x = tf.keras.layers.Dense(2, name='dense_logits')(x)
x = tf.keras.layers.Activation('softmax', dtype='float32', name='predictions')(x)
base_model = tf.keras.Model(inputs=inputs, outputs=x)
base_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image (InputLayer)           [(None, 96, 96, 3)]       0         
_________________________________________________________________
xception (Model)             (None, 3, 3, 2048)        20861480  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense_logits (Dense)         (None, 2)                 4098      
_________________________________________________________________
predictions (Activation)     (None, 2)                 0         
Total params: 20,865,578
Trainable params: 20,811,050
Non-trainable params: 54,528
_________________________________________________________________


In [3]:
tfrecords = [
    '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/tfrecord/stylized_screw_heatsink_before_20200503.tfrecord',
]
def to_ds_comp_label(label, component):
    ds = tf.data.experimental.SqlDataset(
            "sqlite", "/data/robert/p1-dip-metadata.db", 
            f"""select path, label from metadata 
            where label = '{label}' and 
            component_class = '{component}' 
            """, (tf.string, tf.string))
    return ds

ds_list = []
for label in ["NG", "OK"]:
    for component in ["heat_sink", "screw"]:
        ds_list.append(to_ds_comp_label(label, component))
LABEL_CLASS_LIST = ['NG', 'OK']
label_lookup = tf.lookup.StaticHashTable(
        tf.lookup.KeyValueTensorInitializer(LABEL_CLASS_LIST, tf.constant([0, 1], dtype=tf.int64)), -1)
def parse_img(img):
    img = tf.io.decode_image(img, channels=3, dtype=tf.dtypes.float32, expand_animations = False)
#     img = tf.cast(img, dtype=tf.dtypes.float32) / 255.0
    img = tf.image.resize_with_pad(img, 96, 96)
    return img
def label_to_onehot(label):
    label = label_lookup.lookup(label)
    label = tf.one_hot(label, 2)
    # label = tf.cast(label, dtype=tf.float32)
    return label
def parse_example(example_proto):    
    image_feature_description = {
        "path": tf.io.FixedLenFeature([], tf.string),
        "image": tf.io.FixedLenFeature([], tf.string),
        "degree": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.string),
    }
    features_in_example = tf.io.parse_single_example(example_proto, image_feature_description)
    features = {
        'image': parse_img(tf.io.read_file(features_in_example['path'])),
        'label': label_to_onehot(features_in_example['label']),
    }
    label = features['label']
    return features, label

def parse_func(path, label):
    features = {
        'image': parse_img(tf.io.read_file(path)),
        'label': label_to_onehot(label),
    }
    label = features['label']
    return features, label
sampled_tfds = tf.data.TFRecordDataset(tfrecords[0]).map(parse_example, tf.data.experimental.AUTOTUNE).repeat()
ds_from_ds_list = [d.map(parse_func, tf.data.experimental.AUTOTUNE).repeat() for d in ds_list]
ds_from_ds_list.append(sampled_tfds)
print(ds_from_ds_list)

[<RepeatDataset shapes: ({image: (96, 96, None), label: (2,)}, (2,)), types: ({image: tf.float32, label: tf.float32}, tf.float32)>, <RepeatDataset shapes: ({image: (96, 96, None), label: (2,)}, (2,)), types: ({image: tf.float32, label: tf.float32}, tf.float32)>, <RepeatDataset shapes: ({image: (96, 96, None), label: (2,)}, (2,)), types: ({image: tf.float32, label: tf.float32}, tf.float32)>, <RepeatDataset shapes: ({image: (96, 96, None), label: (2,)}, (2,)), types: ({image: tf.float32, label: tf.float32}, tf.float32)>, <RepeatDataset shapes: ({image: (96, 96, None), label: (2,)}, (2,)), types: ({image: tf.float32, label: tf.float32}, tf.float32)>]


In [4]:
sampled_ds = tf.data.experimental.sample_from_datasets(ds_from_ds_list).shuffle(10000)
train_ds = sampled_ds.take(5000).repeat().cache().batch(512).prefetch(tf.data.experimental.AUTOTUNE)
val_ds = sampled_ds.skip(5000).repeat().cache().batch(512).prefetch(tf.data.experimental.AUTOTUNE)

In [4]:
METRICS = [
            tf.keras.metrics.CategoricalAccuracy(name='acc'),
            tf.keras.metrics.Precision(name="precision/ok", class_id=0),
            tf.keras.metrics.Precision(name="precision/ng", class_id=1),
            tf.keras.metrics.Recall(name="recall/ok", class_id=0),
            tf.keras.metrics.Recall(name="recall/ng", class_id=1),
        ]

In [5]:
adv_config = nsl.configs.make_adv_reg_config(multiplier=2e-1, 
                                                adv_step_size=2e-1, 
                                                adv_grad_norm = 'infinity')
model = nsl.keras.AdversarialRegularization(base_model, 
                                                label_keys=['label'], 
                                                adv_config=adv_config)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-4), 
                        loss=tf.keras.losses.CategoricalCrossentropy(),
                        metrics=METRICS)
base_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 1e-4), 
                        loss=tf.keras.losses.CategoricalCrossentropy(),
                        metrics=METRICS)
model

In [7]:
tb_log_dir = '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/tb_logs/test/'
trained_h5_dir = f'''/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/test/'''
callbacks = [
        tf.keras.callbacks.TensorBoard(
            log_dir=tb_log_dir,
            write_graph=False,
            profile_batch=0,
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filepath= trained_h5_dir+'ep_{epoch:03d}-vl_{val_loss:03f}-va_{val_acc:03f}.h5',
            save_best_only=True,
            monitor='val_loss',
            mode='min',
            save_weights_only=True,
        ),
    ]
training_history = model.fit(
                train_ds, 
                epochs=3, 
                validation_data=val_ds, 
                steps_per_epoch=5, 
                validation_steps=10,
                callbacks=callbacks,
                verbose=2 
            )

Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fbbffb394f8> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fbbffb394f8> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7fbbffb394f8> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


5/5 - 49s - loss: 0.5919 - categorical_crossentropy: 0.4645 - acc: 0.8129 - precision/ok: 0.7418 - precision/ng: 0.8688 - recall/ok: 0.8164 - recall/ng: 0.8105 - scaled_adversarial_loss: 0.1274 - val_loss: 0.6249 - val_categorical_crossentropy: 0.4483 - val_acc: 0.8010 - val_precision/ok: 0.7435 - val_precision/ng: 0.8402 - val_recall/ok: 0.7607 - val_recall/ng: 0.8275 - val_scaled_adversarial_loss: 0.1766
Epoch 2/3
5/5 - 56s - loss: 0.1805 - categorical_crossentropy: 0.0937 - acc: 0.9816 - precision/ok: 0.9919 - precision/ng: 0.9753 - recall/ok: 0.9615 - recall/ng: 0.9948 - scaled_adversarial_loss: 0.0867 - val_loss: 0.4712 - val_categorical_crossentropy: 0.3259 - val_acc: 0.8506 - val_precision/ok: 0.8106 - val_precision/ng: 0.8768 - val_recall/ok: 0.8118 - val_recall/ng: 0.8760 - val_scaled_adversarial_loss: 0.1453
Epoch 3/3
5/5 - 30s - loss: 0.0820 - categorical_crossentropy: 0.0211 - acc: 0.9941 - precision/ok: 0.9990 - precision/ng: 0.9911 - recall/ok: 0.9861 - recall/ng: 0.9994 

In [6]:
def test_parse_func(path, degree, label):
    features = {
        'image': parse_img(tf.io.read_file(path)),
        'label': label_to_onehot(label),
    }
    label = features['label']
    return features, label
def test_ds_to_eva(test_ds, batch_size=1024):
    return test_ds.map(test_parse_func, tf.data.experimental.AUTOTUNE).batch(batch_size)
test_ds = tf.data.experimental.SqlDataset(
            'sqlite', '/data/robert/p1-dip-metadata.db', 
            f"""select path, degree, label from metadata 
            where ( label = 'OK' or label = 'NG' ) and 
            ( component_class = 'heat_sink' or
            component_class = 'screw' ) and date >= 20200918
            """, (tf.string, tf.string, tf.string))
print([i for i,_ in enumerate(test_ds)][-1] + 1)
eva_test_ds = test_ds_to_eva(test_ds)

3831


In [10]:
evaluate = model.evaluate(eva_test_ds)
named_results = dict(zip(model.metrics_names, evaluate))
print(named_results)

4/4 [==============================] - 3s 864ms/step - loss: 2.8771 - categorical_crossentropy: 2.7030 - acc: 0.6588 - precision/ok: 0.0061 - precision/ng: 0.9960 - recall/ok: 0.4444 - recall/ng: 0.6598 - scaled_adversarial_loss: 0.1741
{'loss': 2.8770737648010254, 'categorical_crossentropy': 2.7029995918273926, 'acc': 0.6588358283042908, 'precision/ok': 0.006130268331617117, 'precision/ng': 0.9960411787033081, 'recall/ok': 0.4444444477558136, 'recall/ng': 0.6598479151725769, 'scaled_adversarial_loss': 0.1740739345550537}


### load weights

In [9]:
best_for_now = '/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/tune/ep_013-vl_0.154498-va_0.997651.h5'
model.load_weights(best_for_now)
b = model.base_model
b.save_weights('/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/tune_20201022.h5')

In [8]:
import glob
# b = model.base_model
for d in glob.glob('/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/tune/*.h5'):
    print(d)
    model.load_weights(d)
    b = model.base_model
    evaluate = b.evaluate(eva_test_ds)
    named_results = dict(zip(b.metrics_names, evaluate))
    print(named_results)

/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/tune/ep_008-vl_0.151375-va_0.996868.h5
4/4 [==============================] - 1s 306ms/step - loss: 0.0658 - acc: 0.9969 - precision/ok: 0.6071 - precision/ng: 0.9997 - recall/ok: 0.9444 - recall/ng: 0.9971
{'loss': 0.06578820198774338, 'acc': 0.9968676567077637, 'precision/ok': 0.6071428656578064, 'precision/ng': 0.999737024307251, 'recall/ok': 0.9444444179534912, 'recall/ng': 0.9971151351928711}
/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/tune/ep_001-vl_0.962143-va_0.833203.h5
4/4 [==============================] - 2s 544ms/step - loss: 0.3812 - acc: 0.8332 - precision/ok: 0.0245 - precision/ng: 0.9994 - recall/ok: 0.8889 - recall/ng: 0.8329
{'loss': 0.38122883439064026, 'acc': 0.8332028388977051, 'precision/ok': 0.02450229786336422, 'precision/ng': 0.9993706941604614, 'recall/ok': 0.8888888955116272, 'recall/ng': 0.8329399228096008}
/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/tune/ep

In [21]:
evaluate = model.evaluate(eva_test_ds)
named_results = dict(zip(model.metrics_names, evaluate))
print(named_results)

4/4 [==============================] - 3s 819ms/step - loss: 0.8116 - categorical_crossentropy: 0.6763 - acc: 0.9953 - precision/ok: 0.0000e+00 - precision/ng: 0.9953 - recall/ok: 0.0000e+00 - recall/ng: 1.0000 - scaled_adversarial_loss: 0.1353
{'loss': 0.8116016983985901, 'categorical_crossentropy': 0.6763347387313843, 'acc': 0.9953014850616455, 'precision/ok': 0.0, 'precision/ng': 0.9953014850616455, 'recall/ok': 0.0, 'recall/ng': 1.0, 'scaled_adversarial_loss': 0.13526694476604462}


### evaluate / test

In [18]:
b = model.base_model
evaluate = b.evaluate(eva_test_ds)
named_results = dict(zip(b.metrics_names, evaluate))
print(named_results)

4/4 [==============================] - 2s 551ms/step - loss: 2.3997 - acc: 0.5978 - precision/ok: 0.0046 - precision/ng: 0.9952 - recall/ok: 0.3889 - recall/ng: 0.5987
{'loss': 2.3997318744659424, 'acc': 0.5977551341056824, 'precision/ok': 0.004554326646029949, 'precision/ng': 0.9952048659324646, 'recall/ok': 0.3888888955116272, 'recall/ng': 0.5987411737442017}


In [19]:
b.save_weights('/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/test/b.h5')

### load weights

In [8]:
base_model.load_weights('/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/test/base_model.h5')
evaluate = base_model.evaluate(eva_test_ds)
named_results = dict(zip(base_model.metrics_names, evaluate))
print(named_results)

4/4 [==============================] - 2s 574ms/step - loss: 0.3518 - acc: 0.9953 - precision/ok: 0.0000e+00 - precision/ng: 0.9953 - recall/ok: 0.0000e+00 - recall/ng: 1.0000
{'loss': 0.3518054187297821, 'acc': 0.9953014850616455, 'precision/ok': 0.0, 'precision/ng': 0.9953014850616455, 'recall/ok': 0.0, 'recall/ng': 1.0}


In [22]:
evaluate = base_model.evaluate(eva_test_ds)
named_results = dict(zip(base_model.metrics_names, evaluate))
print(named_results)

4/4 [==============================] - 3s 650ms/step - loss: 0.6763 - acc: 0.9953 - precision/ok: 0.0000e+00 - precision/ng: 0.9953 - recall/ok: 0.0000e+00 - recall/ng: 1.0000
{'loss': 0.6762919425964355, 'acc': 0.9953014850616455, 'precision/ok': 0.0, 'precision/ng': 0.9953014850616455, 'recall/ok': 0.0, 'recall/ng': 1.0}


In [ ]:
base_model.load_weights('/data/aoi-wzs-p1-dip-fa-nvidia/label_heatsink_screw/trained_h5/test/b.h5')